# Create Database Objects
## Create schema, warehouse

In [ ]:
USE ROLE ACCOUNTADMIN;
/*--
 • database, schema and warehouse creation
--*/

-- create MAYBANK_DEMO database
CREATE DATABASE IF NOT EXISTS MAYBANK_DEMO;

-- create BRONZE schema
CREATE SCHEMA IF NOT EXISTS MBB_BRONZE;

-- create raw_pos schema
CREATE SCHEMA IF NOT EXISTS MBB_SILVER;

-- create GOLD schema
CREATE SCHEMA IF NOT EXISTS MBB_GOLD;

-- create warehouses
CREATE OR REPLACE WAREHOUSE MBB_WH
    WAREHOUSE_SIZE = 'large' -- Large for initial data load - scaled down to XSmall at end of this scripts
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'data engineering warehouse for Maybank';


CREATE OR REPLACE WAREHOUSE MAYBANK_BI_WH
    WAREHOUSE_SIZE = 'xsmall'
    WAREHOUSE_TYPE = 'standard'
    MIN_CLUSTER_COUNT = 1,
    MAX_CLUSTER_COUNT = 5,
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'BI warehouse for Maybank';

## Create roles and grant permissions to the roles on database objects

In [ ]:
-- create roles
USE ROLE ACCOUNTADMIN;

CREATE ROLE IF NOT EXISTS PII_READER;
CREATE ROLE IF NOT EXISTS ANALYST;
CREATE ROLE IF NOT EXISTS GOVERNANCE;
CREATE ROLE IF NOT EXISTS ENGINEERING;

-- ============================================
-- Database Usage Grants
-- ============================================

GRANT USAGE ON DATABASE MAYBANK_DEMO TO ROLE ANALYST;
GRANT USAGE ON DATABASE MAYBANK_DEMO TO ROLE PII_READER;
GRANT USAGE ON DATABASE MAYBANK_DEMO TO ROLE GOVERNANCE;
GRANT USAGE ON DATABASE MAYBANK_DEMO TO ROLE ENGINEERING;

-- ============================================
-- Warehouse Usage Grants
-- ============================================

GRANT USAGE ON WAREHOUSE MAYBANK_WH TO ROLE ANALYST;
GRANT USAGE ON WAREHOUSE MAYBANK_WH TO ROLE PII_READER;
GRANT USAGE ON WAREHOUSE MAYBANK_WH TO ROLE GOVERNANCE;
GRANT USAGE ON WAREHOUSE MAYBANK_WH TO ROLE ENGINEERING;

-- ============================================
-- Schema Usage Grants
-- ============================================

-- SILVER schema
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE ANALYST;
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE GOVERNANCE;
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE ENGINEERING;

-- BRONZE schema
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_BRONZE TO ROLE PII_READER;
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_BRONZE TO ROLE GOVERNANCE;
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_BRONZE TO ROLE ANALYST;

-- GOLD schema
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_GOLD TO ROLE GOVERNANCE;
GRANT USAGE ON SCHEMA MAYBANK_DEMO.MBB_GOLD TO ROLE ENGINEERING;

## Load Customer with PII Data to Demonstrate PII Data Classification & Tagging Propagation

In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE MAYBANK_DEMO;
USE SCHEMA MBB_BRONZE;

CREATE OR REPLACE TABLE MBB_BRONZE.CUSTOMER_PII
USING TEMPLATE (
  SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
  FROM TABLE(
    INFER_SCHEMA(
      LOCATION => '@MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/customer_pii_0_0_0.snappy.parquet',
      FILE_FORMAT => 'MAYBANK_DEMO.BRONZE.data_fileformat'
    )
  )
);


--Load Customer_PII
COPY INTO MBB_BRONZE.CUSTOMER_PII
FROM @MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/customer_pii_0_0_0.snappy.parquet
FILE_FORMAT = 'MAYBANK_DEMO.BRONZE.data_fileformat'
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

In [ ]:
-- ============================================
-- Masking Policies
-- ============================================

-- PII Mask Policy (VARCHAR)
CREATE OR REPLACE MASKING POLICY MAYBANK_DEMO.MBB_BRONZE.PII_MASK_POLICY
    AS (VAL VARCHAR) RETURNS VARCHAR ->
    CASE
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'PII_READER') THEN VAL
        ELSE '*** MASKED ***'
    END
    COMMENT = 'Mask PII for non-privileged roles';

-- PII Mask Policy (DATE)
CREATE OR REPLACE MASKING POLICY MAYBANK_DEMO.MBB_BRONZE.PII_MASK_POLICY_DATE
    AS (VAL DATE) RETURNS DATE ->
    CASE
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'PII_READER') THEN VAL
        ELSE TO_DATE('1900-01-01')
    END
    COMMENT = 'Mask DATE for non-privileged roles';

-- PII Mask Policy (Quasi-identifier VARCHAR)
CREATE OR REPLACE MASKING POLICY MAYBANK_DEMO.MBB_BRONZE.PII_MASK_POLICY_QUASI_VARCHAR
    AS (VAL VARCHAR) RETURNS VARCHAR ->
    CASE
        WHEN CURRENT_ROLE() IN ('ACCOUNTADMIN', 'PII_READER') THEN VAL
        ELSE '***'
    END
    COMMENT = 'Mask quasi-identifier VARCHARs for non-privileged roles';

-- ============================================
-- Tags
-- ============================================

-- Classification tag
CREATE TAG IF NOT EXISTS MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION
    ALLOWED_VALUES 'PII', 'INTERNAL', 'NON_PII', 'CONFIDENTIAL', 'PUBLIC';

-- Business Domain tag
CREATE TAG IF NOT EXISTS MAYBANK_DEMO.MBB_BRONZE.BUSINESS_DOMAIN
    ALLOWED_VALUES 'CUSTOMER_360', 'BILLING', 'SUPPORT';

-- Data Owner tag
CREATE TAG IF NOT EXISTS MAYBANK_DEMO.MBB_BRONZE.DATA_OWNER
    ALLOWED_VALUES 'DATA_TEAM', 'SECURITY', 'COMPLIANCE';

-- Legal Basis tag
CREATE TAG IF NOT EXISTS MAYBANK_DEMO.MBB_BRONZE.LEGAL_BASIS
    ALLOWED_VALUES 'CONSENT', 'CONTRACT', 'LEGAL_OBLIGATION';

-- Retention Policy tag
CREATE TAG IF NOT EXISTS MAYBANK_DEMO.MBB_BRONZE.RETENTION_POLICY
    ALLOWED_VALUES '12M', '24M', '60M';

-- PDP Category tag (with masking policies)
CREATE TAG IF NOT EXISTS MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY
    ALLOWED_VALUES 'IDENTIFIER', 'CONTACT', 'FINANCIAL', 'SENSITIVE', 'GENERAL';

-- Associate masking policies to PDP_CATEGORY tag
ALTER TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY
    SET MASKING POLICY MAYBANK_DEMO.BRONZE.PII_MASK_POLICY_QUASI_VARCHAR,
        MASKING POLICY MAYBANK_DEMO.BRONZE.PII_MASK_POLICY_DATE;

In [ ]:

-- ============================================
-- Enable Tag Propagation
-- ============================================

ALTER TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION
    SET PROPAGATE = ON_DEPENDENCY_AND_DATA_MOVEMENT, ON_CONFLICT = ALLOWED_VALUES_SEQUENCE;

ALTER TAG MAYBANK_DEMO.MBB_BRONZE.BUSINESS_DOMAIN
    SET PROPAGATE = ON_DEPENDENCY_AND_DATA_MOVEMENT, ON_CONFLICT = ALLOWED_VALUES_SEQUENCE;

ALTER TAG MAYBANK_DEMO.MBB_BRONZE.DATA_OWNER
    SET PROPAGATE = ON_DEPENDENCY_AND_DATA_MOVEMENT, ON_CONFLICT = ALLOWED_VALUES_SEQUENCE;

ALTER TAG MAYBANK_DEMO.MBB_BRONZE.LEGAL_BASIS
    SET PROPAGATE = ON_DEPENDENCY_AND_DATA_MOVEMENT, ON_CONFLICT = ALLOWED_VALUES_SEQUENCE;

ALTER TAG MAYBANK_DEMO.MBB_BRONZE.RETENTION_POLICY
    SET PROPAGATE = ON_DEPENDENCY_AND_DATA_MOVEMENT, ON_CONFLICT = ALLOWED_VALUES_SEQUENCE;

ALTER TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY
    SET PROPAGATE = ON_DEPENDENCY_AND_DATA_MOVEMENT, ON_CONFLICT = ALLOWED_VALUES_SEQUENCE;


-- ============================================
-- Tag Associations - Column Level (CLASSIFICATION)
-- ============================================

-- Strong identifiers / direct PII
ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN CUSTOMER_ID         SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'PII';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN IC_NUMBER           SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'PII';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN FULL_NAME           SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'PII';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN DATE_OF_BIRTH       SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'PII';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN EMAIL_ADDRESS       SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'PII';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN PHONE_NUMBER        SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'PII';

-- Quasi-identifiers / internal
ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN ADDRESS_LINE1       SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN CITY                SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN POSTAL_CODE         SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN GENDER              SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN SEGMENT             SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN RISK_RATING         SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN PRIMARY_BRANCH_CODE SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN PREFERRED_CHANNEL   SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN RELATIONSHIP_START_DATE SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN RESIDENTIAL_COUNTRY SET TAG MAYBANK_DEMO.MBB_BRONZE.CLASSIFICATION = 'INTERNAL';

-- ============================================
-- Tag Associations - Table Level
-- ============================================

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    SET TAG MAYBANK_DEMO.MBB_BRONZE.BUSINESS_DOMAIN = 'CUSTOMER_360';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    SET TAG MAYBANK_DEMO.MBB_BRONZE.DATA_OWNER = 'DATA_TEAM';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    SET TAG MAYBANK_DEMO.MBB_BRONZE.LEGAL_BASIS = 'CONSENT';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    SET TAG MAYBANK_DEMO.MBB_BRONZE.RETENTION_POLICY = '24M';

-- ============================================
-- Tag Associations - Column Level (PDP_CATEGORY)
-- ============================================

-- IDENTIFIER category
ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN IC_NUMBER   SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'IDENTIFIER';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN CUSTOMER_ID SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'IDENTIFIER';

-- CONTACT category
ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN EMAIL_ADDRESS SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'CONTACT';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN PHONE_NUMBER  SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'CONTACT';

-- SENSITIVE category (DOB)
ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN DATE_OF_BIRTH SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'SENSITIVE';

-- GENERAL / quasi identifiers
ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN POSTAL_CODE SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'GENERAL';

ALTER TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
    MODIFY COLUMN CITY        SET TAG MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY = 'GENERAL';

## Stored procedure building silver and gold tables that automatically propagates classifications and tagging

In [ ]:
CREATE OR REPLACE PROCEDURE MAYBANK_DEMO.MBB_SILVER.SP_BUILD_CUSTOMER_PII()
RETURNS STRING
LANGUAGE SQL
AS
$$
BEGIN
    -- Create SILVER.CUSTOMER_PII from BRONZE.CUSTOMER_PII
    CREATE OR REPLACE TABLE MAYBANK_DEMO.MBB_SILVER.CUSTOMER_PII AS
    
with remove_duplicates as
  (
    select
    CUSTOMER_ID,
	IC_NUMBER,
    LOWER(EMAIL_ADDRESS) AS EMAIL_NORM,
    REGEXP_REPLACE(PHONE_NUMBER, '[^0-9]', '') AS PHONE_E164ISH,
    YEAR(DATE_OF_BIRTH) AS BIRTH_YEAR,
	TRIM(FULL_NAME) as Full_Name,
    UPPER(LEFT(GENDER, 1)) as GENDER,
	SEGMENT,
	RISK_RATING,
    CITY,
    POSTAL_CODE,
    RESIDENTIAL_COUNTRY,
	RELATIONSHIP_START_DATE,
    row_number()over(partition by customer_id order by ic_number ) as row_num
    from MBB_BRONZE.CUSTOMER_PII
    )

    select *
    from remove_duplicates
    where row_num = 1;



    -- GOLD view with derived / de-sensitized fields
    CREATE OR REPLACE VIEW MAYBANK_DEMO.MBB_GOLD.CUSTOMER_PII_PUBLIC AS
    SELECT
    CUSTOMER_ID,
    EMAIL_NORM,
    PHONE_E164ISH,
    BIRTH_YEAR,
	Full_Name,
    GENDER,
	SEGMENT,
	RISK_RATING,
    CITY,
    POSTAL_CODE,
    RESIDENTIAL_COUNTRY,
	RELATIONSHIP_START_DATE
    FROM MAYBANK_DEMO.MBB_SILVER.CUSTOMER_PII;

    -- Grants
    GRANT SELECT ON TABLE MAYBANK_DEMO.MBB_SILVER.CUSTOMER_PII TO ROLE ANALYST;
    GRANT SELECT ON TABLE MAYBANK_DEMO.MBB_SILVER.CUSTOMER_PII TO ROLE PII_READER;
    GRANT SELECT ON TABLE MAYBANK_DEMO.MBB_SILVER.CUSTOMER_PII TO ROLE GOVERNANCE;

    GRANT SELECT ON VIEW MAYBANK_DEMO.MBB_GOLD.CUSTOMER_PII_PUBLIC TO ROLE ANALYST;
    GRANT SELECT ON VIEW MAYBANK_DEMO.MBB_GOLD.CUSTOMER_PII_PUBLIC TO ROLE PII_READER;
    GRANT SELECT ON VIEW MAYBANK_DEMO.MBB_GOLD.CUSTOMER_PII_PUBLIC TO ROLE GOVERNANCE;

    RETURN 'SILVER.CUSTOMER_PII built and GOLD.CUSTOMER_PII_PUBLIC created';
END;
$$;

-- Grant USAGE on procedure
GRANT USAGE ON PROCEDURE MAYBANK_DEMO.MBB_SILVER.SP_BUILD_CUSTOMER_PII() TO ROLE GOVERNANCE;
GRANT USAGE ON PROCEDURE MAYBANK_DEMO.MBB_SILVER.SP_BUILD_CUSTOMER_PII() TO ROLE ENGINEERING;

-- ============================================
-- Table SELECT Grants (BRONZE)
-- ============================================

GRANT SELECT ON TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII TO ROLE ANALYST;
GRANT SELECT ON TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII TO ROLE PII_READER;
GRANT SELECT ON TABLE MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII TO ROLE GOVERNANCE;

-- ============================================
-- Future Grants on SILVER Schema
-- ============================================

GRANT SELECT ON FUTURE TABLES IN SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE ANALYST;
GRANT SELECT ON FUTURE TABLES IN SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE PII_READER;
GRANT SELECT ON FUTURE TABLES IN SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE ENGINEERING;
GRANT SELECT ON FUTURE TABLES IN SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE GOVERNANCE;

-- Engineering: CREATE TABLE and CREATE VIEW on SILVER
GRANT CREATE TABLE ON SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE ENGINEERING;
GRANT CREATE VIEW  ON SCHEMA MAYBANK_DEMO.MBB_SILVER TO ROLE ENGINEERING;

-- ============================================
-- Custom Classifier and Profile (Malaysian PDP)
-- ============================================

-- Grant CLASSIFICATION_ADMIN to governance role
GRANT DATABASE ROLE SNOWFLAKE.CLASSIFICATION_ADMIN TO ROLE GOVERNANCE;

-- Create Malaysian PDP custom classifier
CREATE OR REPLACE SNOWFLAKE.DATA_PRIVACY.CUSTOM_CLASSIFIER MAYBANK_DEMO.MBB_BRONZE.MY_PDP();

-- Grant classifier instance role to governance
GRANT SNOWFLAKE.DATA_PRIVACY.CUSTOM_CLASSIFIER ROLE MAYBANK_DEMO.MBB_BRONZE.MY_PDP!PRIVACY_USER
    TO ROLE GOVERNANCE;

-- Malaysian NRIC (12 digits)
CALL MAYBANK_DEMO.MBB_BRONZE.MY_PDP!ADD_REGEX(
    'MALAYSIAN_NRIC_DIGITS',
    'IDENTIFIER',
    '^\\d{12}$',
    NULL,
    'Malaysian National ID (NRIC) digits only',
    0.85
);

-- Malaysian NRIC with dashes YYMMDD-PP-####
CALL MAYBANK_DEMO.MBB_BRONZE.MY_PDP!ADD_REGEX(
    'MALAYSIAN_NRIC_DASH',
    'IDENTIFIER',
    '^\\d{6}-\\d{2}-\\d{4}$',
    NULL,
    'Malaysian NRIC with dashes',
    0.85
);

-- Malaysian mobile phone +601X...
CALL MAYBANK_DEMO.MBB_BRONZE.MY_PDP!ADD_REGEX(
    'MALAYSIAN_PHONE',
    'IDENTIFIER',
    '^\\+601[0-9]\\d{7,8}$',
    NULL,
    'Malaysian mobile phone',
    0.85
);

-- Malaysian postal code (5 digits)
CALL MAYBANK_DEMO.MBB_BRONZE.MY_PDP!ADD_REGEX(
    'POSTAL_CODE_MY',
    'QUASI_IDENTIFIER',
    '^\\d{5}$',
    NULL,
    'Malaysian postal code',
    0.85
);

-- Create classification profile
CREATE OR REPLACE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE MAYBANK_DEMO.MBB_BRONZE.MAYBANK_PROFILE(
    {
        'minimum_object_age_for_classification_days': 0,
        'auto_tag': true,
        'custom_classifiers': {
            'my_pdp': MAYBANK_DEMO.MBB_BRONZE.MY_PDP!LIST()
        }
    }
);



-- Set tag map on profile
CALL MAYBANK_DEMO.MBB_BRONZE.MAYBANK_PROFILE!SET_TAG_MAP(
    {
        'column_tag_map': [
            {
                'tag_name': 'MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY',
                'tag_value': 'IDENTIFIER',
                'semantic_categories': [
                    'MALAYSIAN_NRIC_DIGITS',
                    'MALAYSIAN_NRIC_DASH',
                    'MALAYSIAN_PHONE'
                ]
            },
            {
                'tag_name': 'MAYBANK_DEMO.MBB_BRONZE.PDP_CATEGORY',
                'tag_value': 'GENERAL',
                'semantic_categories': [
                    'POSTAL_CODE_MY'
                ]
            }
        ]
    }
);

-- Set classification profile on BRONZE schema
ALTER SCHEMA MAYBANK_DEMO.MBB_BRONZE
    SET CLASSIFICATION_PROFILE = 'MAYBANK_DEMO.MBB_BRONZE.MAYBANK_PROFILE';

-- Grant APPLY TAG on ACCOUNT to governance
GRANT APPLY TAG ON ACCOUNT TO ROLE GOVERNANCE;

-- Grant EXECUTE AUTO CLASSIFICATION on BRONZE schema
GRANT EXECUTE AUTO CLASSIFICATION ON SCHEMA MAYBANK_DEMO.MBB_BRONZE TO ROLE GOVERNANCE;

-- Grant profile privacy user role to governance
GRANT SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE ROLE MAYBANK_DEMO.MBB_BRONZE.MAYBANK_PROFILE!PRIVACY_USER
    TO ROLE GOVERNANCE;


# Tagging & Masking Policy Automatically Classified and Propagating to Dependent Tables

## 1. Let's query tags applied on columns

In [ ]:
SELECT * 
FROM TABLE(MAYBANK_DEMO.information_schema.TAG_REFERENCES_ALL_COLUMNS('MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII','table'));

## 2. Masking Logic
Masking is tag-driven, not hard-coded per column.
PDP_CATEGORY tag is associated with masking policies
PII_MASK_POLICY for plain VARCHAR.
PII_MASK_POLICY_DATE for dates.
PII_MASK_POLICY_QUASI_VARCHAR for quasi-identifiers.

In [ ]:
DESCRIBE MASKING POLICY MAYBANK_DEMO.MBB_BRONZE.PII_MASK_POLICY_QUASI_VARCHAR;

## 3. Snowflake has built-in classification, but here we’re adding a custom Malaysian PDP classifier
Classifier MY_PDP uses regexes for:
MALAYSIAN_NRIC_DIGITS – 12-digit NRIC.
MALAYSIAN_NRIC_DASH – YYMMDD-PP-####.
MALAYSIAN_PHONE – +601X… mobile numbers.
POSTAL_CODE_MY – 5-digit postcode.
MY_PDP_PROFILE ties those semantic categories to the PDP_CATEGORY tag:
NRIC & phone → PDP_CATEGORY = 'IDENTIFIER'.
Postal code → PDP_CATEGORY = 'GENERAL'.
This means when you run auto-classification, columns can be auto-tagged based on their pattern.

### As ACCOUNTADMIN or PII_READER (full access)
Real values visible (e.g., CUST000123, 900101123456, Ahmad bin Ali, real email/phone). No masking text.

In [ ]:
USE ROLE PII_READER;
SELECT 
    CUSTOMER_ID,
    IC_NUMBER,
    FULL_NAME,
    DATE_OF_BIRTH,
    EMAIL_ADDRESS,
    PHONE_NUMBER,
    ADDRESS_LINE1,
    CITY,
    POSTAL_CODE,
    SEGMENT,
    RISK_RATING
FROM MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
LIMIT 10;

### As ANALYST (restricted view – masked PII)
An analyst should be able to work with the data, but they don’t need exact NRIC or phone numbers.
Same table, same query, different role – masked values appear automatically.

Analysts can still slice and dice by segment, risk rating, city, age bucket (via BIRTH_YEAR from SILVER), but they don’t see identity-level details.
This is centralised in policies and tags, not in application code.

In [ ]:

USE ROLE ANALYST;
SELECT 
    CUSTOMER_ID,
    IC_NUMBER,
    FULL_NAME,
    DATE_OF_BIRTH,
    EMAIL_ADDRESS,
    PHONE_NUMBER,
    ADDRESS_LINE1,
    CITY,
    POSTAL_CODE,
    SEGMENT,
    RISK_RATING
FROM MAYBANK_DEMO.MBB_BRONZE.CUSTOMER_PII
LIMIT 10;


### As ANALYST on GOLD (more consumer-friendly, still safe)
Most analytics use the GOLD layer, which already has some de-sensitisation – plus the same masking still applies if needed.

Analysts are encouraged to use GOLD, which gives them just what they need: segmentation, risk ratings, city, birth year – but no direct PII.

If they go back to BRONZE, masking still protects the sensitive bits.


In [ ]:
USE ROLE GOVERNANCE;
CALL MAYBANK_DEMO.MBB_SILVER.SP_BUILD_CUSTOMER_PII();

### Query SILVER.CUSTOMER_PII table and GOLD.CUSTOMER_PII_PUBLIC view with tags and classification policies applied propagting automatically all the way to GOLD layer.

Show in Horizon Catalog Explorer

In [ ]:
USE ROLE ANALYST;
SELECT *
FROM MAYBANK_DEMO.MBB_GOLD.CUSTOMER_PII_PUBLIC
LIMIT 10;

### As ENGINEERING (build but not read PII fully)

In [ ]:
USE ROLE ENGINEERING;

-- Engineering can create new views/tables in SILVER
CREATE OR REPLACE VIEW MAYBANK_DEMO.SILVER.CUSTOMER_PII_BY_CITY AS
SELECT CITY, COUNT(*) AS CNT
FROM MAYBANK_DEMO.BRONZE.CUSTOMER_PII
GROUP BY CITY;
--Query the view
SELECT * FROM MAYBANK_DEMO.SILVER.CUSTOMER_PII_BY_CITY;

## Data Quality Check Guardrail with Snowflake Data Metric Function

### Check for NULLs in IC_NUMBER column

In [ ]:
use role accountadmin; -- use for simulation
--Below query will return null if executing as ANALYST role - notice this table was not applied for the tagging/classification explicitly, however, Snowflake auto-tagging propagates automatically because the classification applied on the entire schema and thus, IC_NUMBER is a PII data

--CHECK_NULLS: 
Select * from BRONZE.RAW_CUSTOMER where IC_NUMBER IS NULL;

### Check for duplicates in CUSTOMER_ID


In [ ]:
--CHECK_DUPLICATES: 
Select CUSTOMER_ID, Count(*) from BRONZE.RAW_CUSTOMER Group by 1 Having count(*) > 1;

### Logic check for date of birth in future date

In [ ]:
--LOGIC_CHECKS:
Select * from BRONZE.RAW_CUSTOMER WHERE DATE_OF_BIRTH > CURRENT_DATE();

## Handle Data Quality Checks with Snowflake DMF

In [ ]:

ALTER TABLE RAW_CUSTOMER SET DATA_METRIC_SCHEDULE = 'USING CRON 0 8 * * * UTC'; -- run every day 8am UTC

-- Associate DMFs for row count, freshness, null count and add an expectation.
ALTER TABLE RAW_CUSTOMER
  ADD DATA METRIC FUNCTION
   SNOWFLAKE.CORE.ROW_COUNT ON () EXPECTATION Volume_Check (value > 1),  -- Row count (Volume) 
    SNOWFLAKE.CORE.FRESHNESS ON (INGESTION_TIMESTAMP) EXPECTATION lessThan5Mins (VALUE < 300), -- Freshness since last data load
    SNOWFLAKE.CORE.NULL_COUNT ON (IC_NUMBER) EXPECTATION Null_Check (value = 0);  -- Null count on IC Number


  
-- Create a custom DMF for DOB check on customer dimension table at row form
CREATE DATA METRIC FUNCTION INVALID_DOB_FUTURE_DATE_COUNT(
    raw_customer TABLE (date_of_birth DATE, ingestion_timestamp TIMESTAMP_LTZ)
)
RETURNS NUMBER
AS
$$
    SELECT COUNT(*) 
    FROM raw_customer
    WHERE date_of_birth > ingestion_timestamp
$$;

ALTER TABLE raw_customer
  ADD DATA METRIC FUNCTION INVALID_DOB_FUTURE_DATE_COUNT
  ON (date_of_birth, ingestion_timestamp)
  EXPECTATION dobcheck (value=0);



## Show for the DMF dashboard
## Notifications on DMF expectations / failures

## Create Table Schema in Bronze Layer
Data files are staged in Azure Data Lake
Table schemas are automatically created by infering data file schema and file format in Azure Data Lake

In [ ]:
--Bronze table schema creation
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE MAYBANK_WH;
USE DATABASE MAYBANK_DEMO;
USE SCHEMA MBB_BRONZE;

CREATE OR REPLACE TABLE raw_behavior_event
USING TEMPLATE (
  SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
  FROM TABLE(
    INFER_SCHEMA(
      LOCATION => '@MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/behavior_event_0_0_0.snappy.parquet',
      FILE_FORMAT => 'MAYBANK_DEMO.BRONZE.data_fileformat'
    )
  )
);

CREATE OR REPLACE TABLE raw_behavior
USING TEMPLATE (
  SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
  FROM TABLE(
    INFER_SCHEMA(
      LOCATION => '@MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/behavior_0_1_2.snappy.parquet',
      FILE_FORMAT => 'MAYBANK_DEMO.BRONZE.data_fileformat'
    )
  )
);

CREATE OR REPLACE TABLE raw_transactions
USING TEMPLATE (
  SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
  FROM TABLE(
    INFER_SCHEMA(
      LOCATION => '@MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/transactions__0_0_0.snappy.parquet',
      FILE_FORMAT => 'MAYBANK_DEMO.BRONZE.data_fileformat'
    )
  )
);
--Enable schema evolution
ALTER TABLE raw_transactions SET ENABLE_SCHEMA_EVOLUTION = TRUE;

CREATE OR REPLACE TABLE raw_product (
  PRODUCT_ID STRING,
  PRODUCT_NAME STRING,
  CATEGORY STRING,
  INTEREST_TYPE STRING,
  IS_ISLAMIC_FLAG STRING
);

CREATE OR REPLACE TABLE raw_channel
USING TEMPLATE (
  SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
  FROM TABLE(
    INFER_SCHEMA(
      LOCATION => '@MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/channel_0_0_0.snappy.parquet',
      FILE_FORMAT => 'MAYBANK_DEMO.BRONZE.data_fileformat'
    )
  )
);

## Define Table Schema in Silver Layer
Snowflake Dynamic Table Simplifies ELT by streamling ingestion and handling incremental data loading seamlessly with transformation injected as part of table definition

In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE MAYBANK_WH;
USE DATABASE MAYBANK_DEMO;
USE SCHEMA MBB_SILVER;


create or replace dynamic table DIM_PRODUCT
  target_lag = 'DOWNSTREAM'
  warehouse = maybank_wh
  refresh_mode = incremental
  initialize = on_create
  as
  select
  PRODUCT_ID,
  PRODUCT_NAME,
  -- Fix Schema Drift & Nulls
  COALESCE(CATEGORY, 'Uncategorized') AS PRODUCT_CATEGORY,
  -- Standardize Flags
  COALESCE(IS_ISLAMIC_FLAG, 'N/A') AS IS_ISLAMIC_FLAG,
  INTEREST_TYPE
  from mbb_bronze.raw_product;



create or replace dynamic table FACT_TRANSACTION
target_lag = '20 seconds' 
refresh_mode = INCREMENTAL 
initialize = ON_CREATE 
warehouse = MAYBANK_WH
 COMMENT='The table contains records of financial transactions. Each record represents a single transaction and includes details about the customer, product, and financial information.'
 as
WITH unified_transactions AS (
    SELECT
        TRANSACTION_ID,
        CUSTOMER_ID,
        PRODUCT_ID,
        TXN_DATE,
        AMOUNT,
        CURRENCY,
        TXN_TYPE,
        IS_FEE
    FROM bronze.RAW_TRANSACTION_STREAMING

    UNION ALL

    SELECT
        TRANSACTION_ID,
        CUSTOMER_ID,
        PRODUCT_ID,
        TXN_DATE,
        AMOUNT,
        CURRENCY,
        TXN_TYPE,
        IS_FEE
    FROM mbb_bronze.RAW_TRANSACTIONS -- batch data ingested through ADF
)
SELECT
    t.TRANSACTION_ID,
    t.CUSTOMER_ID,
    t.PRODUCT_ID,
    
    -- Cleaning Rule 1: Date Normalization
    t.TXN_DATE AS DATE_VALUE,

    -- Cleaning Rule 2: Logic Correction (Absolute Value)
    ABS(t.AMOUNT) AS TRANSACTION_AMOUNT,

    -- Cleaning Rule 3: Standardization
    COALESCE(UPPER(t.CURRENCY), 'MYR') AS TRANSACTION_CURRENCY,

    -- Cleaning Rule 4: Type Standardization
    INITCAP(t.TXN_TYPE) AS TRANSACTION_TYPE,
    
    -- Cleaning Rule 5: Boolean/Flag Cleanup
    CASE 
        WHEN UPPER(TO_VARCHAR(t.IS_FEE)) IN ('Y', 'YES', '1', 'TRUE') THEN 'Y' 
        ELSE 'N' 
    END AS IS_FEE_FLAG

FROM unified_transactions t;

## Batch Load Raw Data into Bronze Layer from Data Files in Azure Data Lake Storage
### Tables Loaded:
1. RAW_BEHAVIOR_EVENT (Fact)
2. RAW_BEHAVIOR (Dimension)
3. RAW_CHANNEL (Dimension)
4. RAW_PRODUCT (Dimension)
5. RAW_TRANSACTIONS (Fact)

In [ ]:
use database maybank_demo;
use schema mbb_bronze;

--Load Behavior Event
COPY INTO raw_behavior_event
FROM @MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/behavior_event_0_0_0.snappy.parquet
FILE_FORMAT = 'MAYBANK_DEMO.BRONZE.data_fileformat'
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

--Load Behavior with wildcard pattern matching
COPY INTO raw_behavior
FROM @MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE
FILE_FORMAT = (FORMAT_NAME = 'MAYBANK_DEMO.BRONZE.data_fileformat')
PATTERN = '.*behavior_[0-9]+_[0-9]+_[0-9]+\\.snappy\\.parquet'
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

--Load Product
COPY INTO raw_product
FROM @MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/product_0_0_0.csv
FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1 FIELD_OPTIONALLY_ENCLOSED_BY = '"');

--Load Channel
COPY INTO raw_channel
FROM @MAYBANK_DEMO.BRONZE.MAYBANKADLSSTAGE/channel_0_0_0.snappy.parquet
FILE_FORMAT = 'MAYBANK_DEMO.BRONZE.data_fileformat'
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

## Schema Evolution
### Load Transactions (via ADF parameterized pipeline and return for the select statement).
Run the pipeline again for the schema drift and rerun the select statement

In [ ]:
select * from maybank_demo.mbb_bronze.raw_transactions;

## Schema Evolution in JSON Logs
### Log data ingestion directly into Snowflake table without the need to predefine table schema
### DATABASE: LOGS_DB

## Real-time ingestion/streaming data from Kafka Topic through Azure Event Hub
Table: BRONZE.RAW_TRANSACTION_STREAMING
### Sev to show Event Hub

## Near real-time CDC data ingestion from Postgres with Openflow Snowflake connector that supports CDC
### Table: MAYBANK_DEMO.BRONZE.RAW_TRANSACTION_CDC

## REST API Based ingestion
### Table: MAYBANK_DEMO.BRONZE.RAW_PRODUCTS_API

## Federated Cross Cloud Data Query with Iceberg tables
### Data in AWS S3 bucket, and Snowflake instance deployed on Azure SG
### Table: MAYBANK_DEMO.PUBLIC.CUSTOMER_ICEBERG

In [ ]:
SELECT * FROM MAYBANK_DEMO.PUBLIC.CUSTOMER_ICEBERG;

## Explore Ingested Data with Time Travel

In [ ]:
USE ROLE ACCOUNTADMIN;

SELECT COUNT(1) as TimeTravelRowCount
FROM MBB_SILVER.FACT_TRANSACTION;

In [ ]:
SELECT COUNT(1) as TimeTravelRowCount
FROM MBB_GOLD.FACT_TRANSACTION;

In [ ]:
SELECT COUNT(1) as TimeTravelRowCount
FROM MBB_GOLD.FACT_TRANSACTION AT(OFFSET => -180); 

### Query Data as of 3 minutes ago

In [ ]:
SELECT COUNT(1) as TimeTravelRowCount
FROM MBB_SILVER.FACT_TRANSACTION AT(OFFSET => -180); 
-- Retrieves data from 3 minutes ago

## Transforming other Bronze Data tables in Silver Layer with Spark Connect

In [ ]:
from snowflake import snowpark_connect
from snowflake.snowpark.context import get_active_session
session = get_active_session()
print(session)

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

spark = snowpark_connect.server.init_spark_session()

from snowflake.snowpark_connect.resources_initializer import wait_for_resource_initialization
wait_for_resource_initialization()

In [ ]:
raw_channel = spark.read.table("MAYBANK_DEMO.MBB_BRONZE.RAW_CHANNEL")
raw_channel.show()

df_raw_channel = raw_channel.withColumn("CHANNEL_CODE_CLEAN", upper(trim(col("CHANNEL_CODE"))))
    
# 2. Deduplicate: Group by Clean Code, take first non-null attributes
df_raw_channel = df_raw_channel.groupBy("CHANNEL_CODE_CLEAN").agg(
    first("CHANNEL_NAME").alias("CHANNEL_NAME"),
    first("CHANNEL_TYPE").alias("CHANNEL_TYPE"),
    first("DEVICE_TYPE").alias("DEVICE_TYPE"),
    first("LOCATION_TYPE").alias("LOCATION_TYPE"),
    first("IS_24X7_FLAG").alias("RAW_FLAG")
)

# 3. Standardize Flag ('Yes'/'1' -> 'Y')
df_raw_channel = df_raw_channel.withColumn("IS_24X7_FLAG", 
    when(upper(col("RAW_FLAG")).isin("Y", "YES", "TRUE", "1"), "Y").otherwise("N")
).drop("RAW_FLAG").withColumnRenamed("CHANNEL_CODE_CLEAN", "CHANNEL_CODE")

df_raw_channel.show()

df_raw_channel.write.mode("overwrite").saveAsTable("MAYBANK_DEMO.MBB_SILVER.dim_channel")

In [ ]:
raw_behavior_event = spark.read.table("MAYBANK_DEMO.MBB_BRONZE.RAW_BEHAVIOR_EVENT")
raw_behavior_event.withColumn("CAMPAIGN_NAME", trim(regexp_replace(col("CAMPAIGN_NAME"), "Â®", "")))

raw_behavior_event.show()

# Cleaning 2: Normalize Device Type (Raw Agent String -> Category)
df_raw_behavior_event = raw_behavior_event.withColumn("DEVICE_TYPE", 
    when(lower(col("DEVICE_TYPE")).rlike("iphone|android|mobile"), "Mobile")
    .when(lower(col("DEVICE_TYPE")).rlike("windows|mac|desktop"), "Desktop")
    .otherwise(col("DEVICE_TYPE"))
)

# Cleaning 3: Title Case Traffic Source
df_raw_behavior_event = df_raw_behavior_event.withColumn("TRAFFIC_SOURCE", initcap(col("TRAFFIC_SOURCE")))

df_raw_behavior_event.select(
    "BEHAVIOR_EVENT_ID", "BEHAVIOR_EVENT_TYPE", "CHANNEL_CODE", 
    "CONTENT_ID", "CAMPAIGN_ID", "CAMPAIGN_NAME", 
    "DEVICE_TYPE", "TRAFFIC_SOURCE", "JOURNEY_STAGE"
)
df_raw_behavior_event.show()

df_raw_behavior_event.write.mode("overwrite").saveAsTable("MAYBANK_DEMO.MBB_SILVER.dim_behavior_event")

In [ ]:
raw_behavior = spark.read.table("MAYBANK_DEMO.MBB_BRONZE.RAW_BEHAVIOR")

raw_behavior.show()

df_raw_behavior = raw_behavior.select(
        col("BEHAVIOR_RECORD_ID"),
        
        # Cleaning 1: Handle Null Customer (Tag as Anonymous)
        coalesce(col("CUSTOMER_ID"), lit("ANONYMOUS")).alias("CUSTOMER_ID"),
        
        # Cleaning 2: Standardize Join Key (Fixes 'evt_page_casa' mismatch)
        upper(trim(col("BEHAVIOR_EVENT_ID"))).alias("BEHAVIOR_EVENT_ID"),
        
        col("CHANNEL_CODE"),
        
        # Cleaning 3: Parse Chaotic Timestamps -> Date
        # Tries multiple patterns. If all fail, returns Null.
        coalesce(
            to_date(col("RAW_TIMESTAMP"), "yyyy-MM-dd'T'HH:mm:ssXXX"), # ISO Offset
            to_date(col("RAW_TIMESTAMP"), "yyyy-MM-dd'T'HH:mm:ss'Z'"), # ISO UTC
            to_date(col("RAW_TIMESTAMP"), "yyyy-MM-dd HH:mm:ss")       # SQL Standard
        ).alias("DATE_VALUE"),
        
        col("SESSION_ID"),
        col("EVENT_SEQUENCE_IN_SESSION"),
        col("EVENT_VALUE"),
        col("IS_CONVERSION_FLAG"),
        col("CONVERSION_TYPE"),
        
        # Cleaning 4: Cast Score to Float (Handles 'N/A', 'invalid', 'Error')
        # Casting a non-numeric string to float results in Null in Spark
        col("EXPERIENCE_SCORE").cast("float").alias("EXPERIENCE_SCORE")
    )
    
# Data Quality Filter: Drop rows where Date could not be parsed (Junk Data)
df_raw_behavior_clean = df_raw_behavior.filter(col("DATE_VALUE").isNotNull())

df_raw_behavior_clean.show()

df_raw_behavior_clean.write.mode("overwrite").saveAsTable("MAYBANK_DEMO.MBB_SILVER.fact_behavior")

## Gold Layer Creation

In [ ]:
create or replace dynamic table MBB_GOLD.DIM_BEHAVIOR_EVENT
target_lag = 'DOWNSTREAM'
warehouse = Maybank_wh
refresh_mode = incremental
initialize = on_create
as
select *
from mbb_silver.dim_behavior_event;


create or replace dynamic table MBB_GOLD.DIM_CHANNEL
target_lag = 'DOWNSTREAM'
warehouse = Maybank_wh
refresh_mode = incremental
initialize = on_create
as
select *
from mbb_silver.dim_channel;


create or replace dynamic table MBB_GOLD.FACT_BEHAVIOR
target_lag = 'DOWNSTREAM'
warehouse = Maybank_wh
refresh_mode = incremental
initialize = on_create
as
select *
from mbb_silver.fact_behavior;

create or replace dynamic table MBB_GOLD.DIM_PRODUCT
target_lag = 'DOWNSTREAM'
warehouse = Maybank_wh
refresh_mode = incremental
initialize = on_create
as
select *
from mbb_silver.DIM_PRODUCT;

--Create fact_transactions with integrtity check to avoid any arriving facts on customer_id
create or replace dynamic table MBB_GOLD.FACT_TRANSACTION
target_lag = '30 seconds'
warehouse = Maybank_wh
refresh_mode = incremental
initialize = on_create
as
select t.*
from mbb_silver.FACT_TRANSACTION t
INNER JOIN mbb_silver.CUSTOMER_PII c 
    ON t.CUSTOMER_ID = c.CUSTOMER_ID
WHERE 
    t.TRANSACTION_ID IS NOT NULL
    AND t.TRANSACTION_AMOUNT IS NOT NULL
    AND t.DATE_VALUE BETWEEN '2000-01-01' AND CURRENT_DATE();

## Automatic column description with Cortex AI

In [ ]:
CALL MAYBANK_DEMO.PUBLIC.AUTO_DESCRIBE_OBJECTS('MAYBANK_DEMO', 'MBB_BRONZE', true, true);
CALL MAYBANK_DEMO.PUBLIC.AUTO_DESCRIBE_OBJECTS('MAYBANK_DEMO', 'MBB_SILVER', true, true);
CALL MAYBANK_DEMO.PUBLIC.AUTO_DESCRIBE_OBJECTS('MAYBANK_DEMO', 'MBB_GOLD', true, true);

## Snowflake Multi-cluster scaling

In [ ]:
CREATE OR REPLACE PROCEDURE MAYBANK_DEMO.MBB_GOLD.EXECUTE_BI_QUERIES_IN_PARALLELL(
    sql_list ARRAY,
    parallell_degree INT
)
RETURNS ARRAY
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
PACKAGES = ('snowflake-snowpark-python', 'requests')
HANDLER = 'execute_in_parallel'
AS
$$
from concurrent.futures import ThreadPoolExecutor
from snowflake.snowpark import Session

def execute_in_parallel(session: Session, sql_list, parallell_degree):

    def run_query(sql_command):
        try:
            session.sql(sql_command).collect()
            return f"OK: {sql_command}"
        except Exception as e:
            return f"ERROR in: {sql_command} -> {str(e)}"

    results = []
    with ThreadPoolExecutor(max_workers=parallell_degree) as executor:
        futures = [executor.submit(run_query, sql_command) for sql_command in sql_list]
        for future in futures:
            results.append(future.result())
    return results
$$;

In [ ]:
use warehouse MAYBANK_BI_WH;

CALL MAYBANK_DEMO.MBB_GOLD.EXECUTE_BI_QUERIES_IN_PARALLELL(
    ARRAY_CONSTRUCT(
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)',
        'select * from 
(
select * from 
MAYBANK_DEMO.MBB_GOLD.customer_pii_public c
inner join MAYBANK_DEMO.MBB_GOLD.fact_transaction t on t.customer_id = c.customer_id
inner join MAYBANK_DEMO.MBB_GOLD.dim_product p on t.product_id = p.product_id
)
SAMPLE (1)'
    ),
    10 -- Parallelism
);